# Value Based Drafting (VBD) Analysis

This notebook calculates VBD using multiple industry-standard methods:
- **VOLS** (Value Over Like Starters): teams × starters
- **VORP** (Value Over Replacement Player): teams × (starters + 1)
- **BEER** (Best Eleven Every Round): teams × (starters + 0.5)
- **Blended**: Weighted combination of all methods

**Input**: Processed projections from `02_basic_rankings.ipynb`
**Output**: Top 300 players ranked by VBD methods

In [ ]:
import sys
import os
import pandas as pd
import numpy as np

# Add src directory to path
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

from vbd import calculate_all_vbd_methods, get_top_players_by_vbd, analyze_vbd_distribution
from scoring import load_league_config
from utils import setup_logging, save_rankings, compare_rankings
import logging

# Setup logging
setup_logging()

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Load Processed Data

In [ ]:
# Load the most recent processed rankings
try:
    # Look for most recent rankings file
    rankings_files = [f for f in os.listdir('../../data/output') if f.startswith('rankings_top300_')]
    if not rankings_files:
        raise FileNotFoundError("No rankings files found. Run 02_basic_rankings.ipynb first.")
    
    latest_file = sorted(rankings_files)[-1]
    df = pd.read_csv(f'../../data/output/{latest_file}')
    
    print(f"Loaded {len(df)} players from {latest_file}")
    print(f"Columns: {list(df.columns)}")
    
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please run 02_basic_rankings.ipynb first to generate processed projections.")

## Load League Configuration

In [ ]:
# Load league settings
config = load_league_config('../../config/league-config.yaml')

print("League Configuration:")
print(f"Teams: {config.get('basic_settings', {}).get('teams', 'Not specified')}")
print(f"Roster slots: {config.get('roster', {}).get('roster_slots', {})}")

# Show baseline calculations
teams = config.get('basic_settings', {}).get('teams', 12)
roster_slots = config.get('roster', {}).get('roster_slots', {})

print("\nBaseline Calculations (players at replacement level):")
for pos, starters in roster_slots.items():
    vols = teams * starters
    vorp = teams * (starters + 1)
    beer = int(teams * (starters + 0.5))
    print(f"{pos}: VOLS={vols}, VORP={vorp}, BEER={beer}")

## Calculate VBD Methods

In [ ]:
# Calculate all VBD methods
df_vbd = calculate_all_vbd_methods(df, config)

print(f"VBD calculations complete for {len(df_vbd)} players")
print("\nVBD columns added:")
vbd_cols = [col for col in df_vbd.columns if col.startswith('VBD_')]
print(vbd_cols)

## Top Players by Each VBD Method

In [ ]:
# Show top 20 players by each VBD method
methods = ['VBD_VOLS', 'VBD_VORP', 'VBD_BEER', 'VBD_BLENDED']

for method in methods:
    print(f"\n=== TOP 20 PLAYERS BY {method} ===")
    top_20 = df_vbd.nlargest(20, method)[['PLAYER', 'Position', 'FANTASY_PTS', method]].reset_index(drop=True)
    top_20.index += 1  # Start ranking from 1
    display(top_20)

## VBD Method Comparison

In [ ]:
# Compare how different methods rank the same players
comparison = compare_rankings(df_vbd, methods=['VBD_VOLS', 'VBD_VORP', 'VBD_BEER', 'VBD_BLENDED'], top_n=30)

print("Comparison of Top 30 Rankings Across VBD Methods:")
pivot_comparison = comparison.pivot_table(
    index=['PLAYER', 'Position'], 
    columns='Method', 
    values='Rank', 
    fill_value='>30'
)

display(pivot_comparison.head(20))

## Position Distribution Analysis

In [ ]:
# Analyze position distribution in top 300 by blended VBD
distribution = analyze_vbd_distribution(df_vbd, top_n=300)

print("Position Distribution in Top 300 (Blended VBD):")
display(distribution)

# Show distribution by different VBD methods
print("\nPosition Distribution Comparison Across Methods:")
for method in ['VBD_VOLS', 'VBD_VORP', 'VBD_BEER', 'VBD_BLENDED']:
    top_300 = get_top_players_by_vbd(df_vbd, method=method, top_n=300)
    dist = top_300['Position'].value_counts()
    print(f"\n{method}:")
    for pos, count in dist.items():
        print(f"  {pos}: {count} ({count/300*100:.1f}%)")

## Filter to Top 300 and Save

In [ ]:
# Get top 300 players by blended VBD
top_300_vbd = get_top_players_by_vbd(df_vbd, method='VBD_BLENDED', top_n=300)

# Add overall VBD rank
top_300_vbd['VBD_RANK'] = range(1, len(top_300_vbd) + 1)

print(f"Top 300 players selected by Blended VBD")
print(f"VBD range: {top_300_vbd['VBD_BLENDED'].max():.2f} to {top_300_vbd['VBD_BLENDED'].min():.2f}")

# Show final rankings
display_cols = ['VBD_RANK', 'PLAYER', 'Position', 'FANTASY_PTS', 'VBD_BLENDED', 'VBD_VOLS', 'VBD_VORP', 'VBD_BEER']
print("\nTop 20 by Blended VBD:")
display(top_300_vbd[display_cols].head(20))

## Save VBD Rankings

In [ ]:
# Save VBD rankings to output
filename = save_rankings(top_300_vbd, filename_prefix="vbd_rankings_top300", output_path="../../data/output")
print(f"VBD rankings saved to: {filename}")

# Also save separate files for each method
for method in ['VBD_VOLS', 'VBD_VORP', 'VBD_BEER']:
    method_top300 = get_top_players_by_vbd(df_vbd, method=method, top_n=300)
    method_filename = save_rankings(
        method_top300, 
        filename_prefix=f"rankings_{method.lower()}_top300", 
        output_path="../../data/output"
    )
    print(f"{method} rankings saved to: {method_filename}")

print("\n✅ VBD analysis complete!")
print("Next steps:")
print("- Use blended VBD rankings for draft preparation")
print("- Compare different methods for positional strategy")
print("- Run 03_draft_strategy.ipynb for draft board analysis")